<a href="https://colab.research.google.com/github/KarthikTummala18/INFO-5731-Computational-Methods-for-Information-Systems-/blob/main/Karthik_Tummala_Assignmet_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment 2**

In this assignment, you will work on gathering text data from an open data source via web scraping or API. Following this, you will need to clean the text data and perform syntactic analysis on the data. Follow the instructions carefully and design well-structured Python programs to address each question.

**Expectations**:
*   Use the provided .*ipynb* document to write your code & respond to the questions. Avoid generating a new file.
*   Write complete answers and run all the cells before submission.
*   Make sure the submission is "clean"; *i.e.*, no unnecessary code cells.
*   Once finished, allow shared rights from top right corner (*see Canvas for details*).

* **Make sure to submit the cleaned data CSV in the comment section - 10 points**

**Total points**: 100

**Deadline**: Monday, at 11:59 PM.

**Late Submission will have a penalty of 10% reduction for each day after the deadline.**

**Please check that the link you submitted can be opened and points to the correct assignment.**


# Question 1 (25 points)

Write a python program to collect text data from **either of the following sources** and save the data into a **csv file:**

(1) Collect all the customer reviews of a product (you can choose any porduct) on amazon. [atleast 1000 reviews]

(2) Collect the top 1000 User Reviews of a movie recently in 2023 or 2024 (you can choose any movie) from IMDB. [If one movie doesn't have sufficient reviews, collect reviews of atleast 2 or 3 movies]


(3) Collect the **abstracts** of the top 10000 research papers by using the query "machine learning", "data science", "artifical intelligence", or "information extraction" from Semantic Scholar.

(4) Collect all the information of the 904 narrators in the Densho Digital Repository.

(5)**Collect a total of 10000 reviews** of the top 100 most popular software from G2 and Capterra.


In [ ]:
import csv
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import json
import re


main_url = "https://ddr.densho.org/narrators/?page={}"


narrators = []


for page_num in range(1, 41):
    link1 = Request(main_url.format(page_num), headers={'User-Agent': 'Mozilla/5.0'})
    url1 = urlopen(link1)
    data1 = url1.read()
    data1_soup = BeautifulSoup(data1, 'html.parser')


    narrators_list = data1_soup.find_all('h4', class_='media-heading')

    for narrator in narrators_list:

        narrator_link = narrator.find('a')['href']
        narrator_url = narrator_link


        link2 = Request(narrator_url, headers={'User-Agent': 'Mozilla/5.0'})
        url2 = urlopen(link2)
        data2 = url2.read()
        data2_soup = BeautifulSoup(data2, 'html.parser')
        narrator_name = data2_soup.find('h1').get_text(strip=True)
        bio = data2_soup.find('p').get_text(strip=True)
        interview_divs = data2_soup.find_all('div', class_='media-body')

        for div in interview_divs:
            interview_title = div.find('b', class_='media-heading').get_text(strip=True)
            interview_details = div.find('div', class_='source muted').get_text(strip=True)
            interview_link = div.find('div', class_='url').get_text(strip=True)
            narrators.append({
                'name': narrator_name,
                'bio': bio,
                'interview_title': interview_title,
                'interview_details': interview_details,
                'interview_link': interview_link,
                'narrator_url': narrator_url
            })


csv_filename = 'narrators_data.csv'


with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['name', 'bio', 'interview_title', 'interview_details', 'interview_link', 'narrator_url']

    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for narrator in narrators:
        writer.writerow({
            'name': narrator['name'],
            'bio': narrator['bio'],
            'interview_title': narrator['interview_title'],
            'interview_details': narrator['interview_details'],
            'interview_link': narrator['interview_link'],
            'narrator_url': narrator['narrator_url']
        })


print(f"Total narrators collected: {len(narrators)}")

Total narrators collected: 1183


# Question 2 (15 points)

Write a python program to **clean the text data** you collected in the previous question and save the clean data in a new column in the csv file. The data cleaning steps include: [Code and output is required for each part]

(1) Remove noise, such as special characters and punctuations.

(2) Remove numbers.

(3) Remove stopwords by using the stopwords list.

(4) Lowercase all texts

(5) Stemming.

(6) Lemmatization.

In [ ]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:

df = pd.read_csv('narrators_data.csv')

df['bio'] = df['bio'].fillna('')
df['interview_details'] = df['interview_details'].fillna('')

def remove_noise(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

def remove_numbers(text):
    text = re.sub(r'\d+', '', text)
    return text

stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    text = " ".join([word for word in text.split() if word.lower() not in stop_words])
    return text

def to_lowercase(text):
    if isinstance(text, str):
        return text.lower()
    return text

stemmer = PorterStemmer()
def stemming(text):
    text = " ".join([stemmer.stem(word) for word in text.split()])
    return text

lemmatizer = WordNetLemmatizer()
def lemmatization(text):
    text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
    return text


In [ ]:

df['cleaned_name'] = df['name'].apply(to_lowercase)
df['cleaned_bio'] = df['bio'].apply(remove_noise)
df['cleaned_bio'] = df['cleaned_bio'].apply(remove_numbers)
df['cleaned_bio'] = df['cleaned_bio'].apply(remove_stopwords)
df['cleaned_bio'] = df['cleaned_bio'].apply(to_lowercase)
df['cleaned_bio'] = df['cleaned_bio'].apply(stemming)
df['cleaned_bio'] = df['cleaned_bio'].apply(lemmatization)

df['cleaned_interview_details'] = df['interview_details'].apply(remove_noise)
df['cleaned_interview_details'] = df['cleaned_interview_details'].apply(remove_numbers)
df['cleaned_interview_details'] = df['cleaned_interview_details'].apply(remove_stopwords)
df['cleaned_interview_details'] = df['cleaned_interview_details'].apply(to_lowercase)
df['cleaned_interview_details'] = df['cleaned_interview_details'].apply(stemming)
df['cleaned_interview_details'] = df['cleaned_interview_details'].apply(lemmatization)

df.to_csv('narrators_data_cleaned.csv', index=False)

print(df[['name', 'cleaned_name', 'bio', 'cleaned_bio', 'interview_details','cleaned_interview_details']].head(2))


           name  cleaned_name  \
0  Kay Aiko Abe  kay aiko abe   
1       Art Abe       art abe   

                                                 bio  \
0  Nisei female. Born May 9, 1927, in Selleck, Wa...   
1  Nisei male. Born June 12, 1921, in Seattle, Wa...   

                                         cleaned_bio  \
0  nisei femal born may selleck washington spent ...   
1  nisei male born june seattl washington grew ar...   

                                   interview_details  \
0  December 2, 2008.\n      Seattle, Washington.0...   
1  January 24, 2008.\n      Seattle, Washington.0...   

           cleaned_interview_details  
0   decemb seattl washington segment  
1  januari seattl washington segment  


In [ ]:
df = pd.read_csv('/content/narrators_data_cleaned.csv')
print(df.head(2))

           name                                                bio  \
0  Kay Aiko Abe  Nisei female. Born May 9, 1927, in Selleck, Wa...   
1       Art Abe  Nisei male. Born June 12, 1921, in Seattle, Wa...   

                                interview_title  \
0  Kay Aiko Abe Interview — ddr-densho-1000-232   
1       Art Abe Interview — ddr-densho-1000-206   

                                   interview_details  \
0  December 2, 2008.\n      Seattle, Washington.0...   
1  January 24, 2008.\n      Seattle, Washington.0...   

                                interview_link  \
0  https://ddr.densho.org/ddr-densho-1000-232/   
1  https://ddr.densho.org/ddr-densho-1000-206/   

                            narrator_url  cleaned_name  \
0  https://ddr.densho.org/narrators/361/  kay aiko abe   
1  https://ddr.densho.org/narrators/291/       art abe   

                                         cleaned_bio  \
0  nisei femal born may selleck washington spent ...   
1  nisei male born june seat

# Question 3 (15 points)

Write a python program to **conduct syntax and structure analysis of the clean text** you just saved above. The syntax and structure analysis includes:

(1) **Parts of Speech (POS) Tagging:** Tag Parts of Speech of each word in the text, and calculate the total number of N(oun), V(erb), Adj(ective), Adv(erb), respectively.

(2) **Constituency Parsing and Dependency Parsing:** print out the constituency parsing trees and dependency parsing trees of all the sentences. Using one sentence as an example to explain your understanding about the constituency parsing tree and dependency parsing tree.

(3) **Named Entity Recognition:** Extract all the entities such as person names, organizations, locations, product names, and date from the clean texts, calculate the count of each entity.

In [ ]:
import spacy
import pandas as pd
from collections import Counter
from spacy import displacy


nlp = spacy.load("en_core_web_sm")


df = pd.read_csv('/content/narrators_data_cleaned.csv')

bio = df['bio'].dropna()
cleaned_bio = df['cleaned_bio'].dropna()

df['original_noun_count'] = 0
df['original_verb_count'] = 0
df['original_adj_count'] = 0
df['original_adv_count'] = 0

df['cleaned_noun_count'] = 0
df['cleaned_verb_count'] = 0
df['cleaned_adj_count'] = 0
df['cleaned_adv_count'] = 0

for index, (original_text, cleaned_text) in enumerate(zip(bio, cleaned_bio)):
    noun_count = 0
    verb_count = 0
    adj_count = 0
    adv_count = 0

    cleaned_noun_count = 0
    cleaned_verb_count = 0
    cleaned_adj_count = 0
    cleaned_adv_count = 0

    doc = nlp(original_text)

    for token in doc:
        if token.pos_ == "NOUN":
            noun_count += 1
        elif token.pos_ == "VERB":
            verb_count += 1
        elif token.pos_ == "ADJ":
            adj_count += 1
        elif token.pos_ == "ADV":
            adv_count += 1

    cleaned_doc = nlp(cleaned_text)

    for token in cleaned_doc:
        if token.pos_ == "NOUN":
            cleaned_noun_count += 1
        elif token.pos_ == "VERB":
            cleaned_verb_count += 1
        elif token.pos_ == "ADJ":
            cleaned_adj_count += 1
        elif token.pos_ == "ADV":
            cleaned_adv_count += 1

    df.at[index, 'original_noun_count'] = noun_count
    df.at[index, 'original_verb_count'] = verb_count
    df.at[index, 'original_adj_count'] = adj_count
    df.at[index, 'original_adv_count'] = adv_count

    df.at[index, 'cleaned_noun_count'] = cleaned_noun_count
    df.at[index, 'cleaned_verb_count'] = cleaned_verb_count
    df.at[index, 'cleaned_adj_count'] = cleaned_adj_count
    df.at[index, 'cleaned_adv_count'] = cleaned_adv_count

print(df[['bio',  'original_noun_count', 'original_verb_count', 'original_adj_count', 'original_adv_count', 'cleaned_bio',
          'cleaned_noun_count', 'cleaned_verb_count', 'cleaned_adj_count', 'cleaned_adv_count']].head())

df.to_csv('updated_narrators_data.csv', index=False)

                                                 bio  original_noun_count  \
0  Nisei female. Born May 9, 1927, in Selleck, Wa...                   12   
1  Nisei male. Born June 12, 1921, in Seattle, Wa...                   16   
2  Nisei female. Born October 31, 1925, in Seattl...                   10   
3  Nisei female. Born October 31, 1925, in Seattl...                   10   
4  Nisei female. Born July 8, 1928, in Boyle Heig...                   13   

   original_verb_count  original_adj_count  original_adv_count  \
0                    8                   4                   0   
1                   14                   8                   2   
2                    8                   3                   2   
3                    8                   3                   2   
4                   10                   3                   2   

                                         cleaned_bio  cleaned_noun_count  \
0  nisei femal born may selleck washington spent ...            

In [ ]:
import spacy
import pandas as pd
from spacy import displacy


nlp = spacy.load("en_core_web_sm")


df = pd.read_csv('updated_narrators_data.csv')

def print_parsing_trees(text, index):
    doc = nlp(text)


    displacy.render(doc, style="dep", jupyter=False, page=True, options={"compact": True})

    output_path = f"dependency_tree_{index}.svg"
    displacy.serve(doc, style="dep", options={"compact": True}, port=5000)
    print(f"Dependency Parsing tree for: {text}")
    print(f"Tree served at: http://localhost:5000/\n")

for index, bio in enumerate(df['bio'][:3]):
    print_parsing_trees(bio, index)

/usr/local/lib/python3.11/dist-packages/spacy/displacy/__init__.py:106: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
Dependency Parsing tree for: Nisei female. Born May 9, 1927, in Selleck, Washington. Spent much of childhood in Beaverton, Oregon, where father owned a farm. Influenced at an early age by parents' conversion to Christianity. During World War II, removed to the Portland Assembly Center, Oregon, and the Minidoka concentration camp, Idaho. After the war, worked to establish a successful volunteer program to feed the homeless in Seattle, Washington.
Tree served at: http://localhost:5000/




Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.
Dependency Parsing tree for: Nisei male. Born June 12, 1921, in Seattle, Washington. Grew up in an area of Seattle with few other Japanese Americans, and was attending the University of Washington when Japan bombed Pearl Harbor. Removed to the Puyallup Assembly Center, Washington, and the Minidoka concentration camp, Idaho. Left Minidoka several times on temporary work leave to work on farms in the area. Suffered tragic loss in camp when father got lost outside collecting wood and perished in the elements. Volunteered for the army and served in the signal corps, eventually working for the Civil Aeronautics Administration and Boeing. Also one of the early members of the Seattle Nisei Veterans Committee.
Tree served at: http://localhost:5000/




Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...



In [ ]:
df = pd.read_csv('/content/narrators_data_cleaned.csv')


bio = df['bio'].dropna()
cleaned_bio = df['cleaned_bio'].dropna()


df['person_count'] = 0
df['organization_count'] = 0
df['location_count'] = 0
df['product_count'] = 0
df['date_count'] = 0


for index, (original_text, cleaned_text) in enumerate(zip(bio, cleaned_bio)):

    person_count = 0
    organization_count = 0
    location_count = 0
    product_count = 0
    date_count = 0


    doc = nlp(original_text)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            person_count += 1
        elif ent.label_ == 'ORG':
            organization_count += 1
        elif ent.label_ == 'GPE':
            location_count += 1
        elif ent.label_ == 'PRODUCT':
            product_count += 1
        elif ent.label_ == 'DATE':
            date_count += 1


    cleaned_doc = nlp(cleaned_text)
    for ent in cleaned_doc.ents:
        if ent.label_ == 'PERSON':
            person_count += 1
        elif ent.label_ == 'ORG':
            organization_count += 1
        elif ent.label_ == 'GPE':
            location_count += 1
        elif ent.label_ == 'PRODUCT':
            product_count += 1
        elif ent.label_ == 'DATE':
            date_count += 1


    df.at[index, 'person_count'] = person_count
    df.at[index, 'organization_count'] = organization_count
    df.at[index, 'location_count'] = location_count
    df.at[index, 'product_count'] = product_count
    df.at[index, 'date_count'] = date_count

print(df[['bio', 'cleaned_bio', 'person_count', 'organization_count', 'location_count', 'product_count', 'date_count']].head(3))


df.to_csv('updated_narrators_data_with_entities.csv', index=False)



                                                 bio  \
0  Nisei female. Born May 9, 1927, in Selleck, Wa...   
1  Nisei male. Born June 12, 1921, in Seattle, Wa...   
2  Nisei female. Born October 31, 1925, in Seattl...   

                                         cleaned_bio  person_count  \
0  nisei femal born may selleck washington spent ...             2   
1  nisei male born june seattl washington grew ar...             4   
2  nisei femal born octob seattl washington famil...             2   

   organization_count  location_count  product_count  date_count  
0                   2              12              0           1  
1                   8              11              0           2  
2                   3              12              0           2  


# **Following Questions must answer using AI assitance**

#Question 4 (20 points).

Q4. (PART-1)
Web scraping data from the GitHub Marketplace to gather details about popular actions. Using Python, the process begins by sending HTTP requests to multiple pages of the marketplace (1000 products), handling pagination through dynamic page numbers. The key details extracted include the product name, a short description, and the URL.

 The extracted data is stored in a structured CSV format with columns for product name, description, URL, and page number. A time delay is introduced between requests to avoid server overload. ChatGPT can assist by helping with the parsing of HTML, error handling, and generating reports based on the data collected.

 The goal is to complete the scraping within a specified time limit, ensuring that the process is efficient and adheres to GitHub’s usage guidelines.

(PART -2)

1.   **Preprocess Data**: Clean the text by tokenizing, removing stopwords, and converting to lowercase.

2. Perform **Data Quality** operations.


Preprocessing:
Preprocessing involves cleaning the text by removing noise such as special characters, HTML tags, and unnecessary whitespace. It also includes tasks like tokenization, stopword removal, and lemmatization to standardize the text for analysis.

Data Quality:
Data quality checks ensure completeness, consistency, and accuracy by verifying that all required columns are filled and formatted correctly. Additionally, it involves identifying and removing duplicates, handling missing values, and ensuring the data reflects the true content accurately.


Github MarketPlace page:
https://github.com/marketplace?type=actions

Part - 1

In [ ]:
import csv
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import time
import random
import json
import re


main_url = "https://github.com/marketplace?type=actions&page={}"


products_info = []


for page_num in range(1, 55):
    link1 = Request(main_url.format(page_num), headers={'User-Agent': 'Mozilla/5.0'})

    retry_count = 0
    while retry_count < 5:
        try:
            url1 = urlopen(link1)
            data1 = url1.read()
            data1_soup = BeautifulSoup(data1, 'html.parser')
            break
        except Exception as e:
            print(f"Error occurred while fetching page {page_num}: {e}")
            retry_count += 1
            time.sleep(random.uniform(2, 5))
    else:
        print(f"Failed to retrieve page {page_num} after multiple attempts.")
        continue


    products = data1_soup.find_all('div', class_="d-flex flex-justify-between flex-items-start gap-3")

    for product in products:

        name_tag = product.find('h3', class_="d-flex f4 lh-condensed prc-Heading-Heading-6CmGO")
        name = name_tag.get_text(strip=True) if name_tag else "No name available."


        url_tag = product.find('a')
        url = f"https://github.com{url_tag['href']}" if url_tag else None

        description_text = "No description available."
        if url:
            retry_count = 0
            while retry_count < 5:
                try:
                    link2 = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
                    url2 = urlopen(link2)
                    data2 = url2.read()
                    data2_soup = BeautifulSoup(data2, 'html.parser')

                    description_taga = data2_soup.find('div', class_="Box-sc-g0xbh4-0 gpEbNC")
                    description_tagb = description_taga.find('div', class_="Stack__StyledStack-sc-x3xa2i-0 hyhjyo") if description_taga else None
                    description_tag = description_tagb.find('span') if description_tagb else None
                    description_text = description_tag.get_text() if description_tag else "No description available."
                    break
                except Exception as e:
                    print(f"Error occurred while fetching description for {url}: {e}")
                    retry_count += 1
                    time.sleep(random.uniform(2, 5))
            else:
                print(f"Failed to retrieve description for {url} after multiple attempts.")

        products_info.append({
            'name': name,
            'url': url,
            'description': description_text,
            'page_number': page_num
        })

    time.sleep(random.uniform(3, 6))


with open('products_info.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=['name', 'url', 'description', 'page_number'])
    writer.writeheader()
    writer.writerows(products_info)


In [ ]:
df = pd.read_csv('/content/products_info.csv')
print(df.head(30))

Part - 2

Data Preprocessing

In [ ]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def preprocess(text):

    text = re.sub(r'<.*?>', '', text)

    text = re.sub(r'[^A-Za-z\s]', '', text)

    text = text.lower()

    tokens = text.split()

    stop_words = set(stopwords.words('english'))
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)


df['cleaned_name'] = df['name'].apply(preprocess)
df['cleaned_description'] = df['description'].apply(preprocess)
df['cleaned_url'] = df['url'].apply(preprocess)

print(df[['name', 'cleaned_name', 'url', 'cleaned_url','description','cleaned_description']].head())


                           name                cleaned_name  \
0                TruffleHog OSS               trufflehog os   
1                 Metrics embed                metric embed   
2  yq - portable yaml processor  yq portable yaml processor   
3                  Super-Linter                 superlinter   
4        Gosec Security Checker      gosec security checker   

                                                 url  \
0  https://github.com/marketplace/actions/truffle...   
1  https://github.com/marketplace/actions/metrics...   
2  https://github.com/marketplace/actions/yq-port...   
3  https://github.com/marketplace/actions/super-l...   
4  https://github.com/marketplace/actions/gosec-s...   

                                         cleaned_url  \
0      httpsgithubcommarketplaceactionstrufflehogoss   
1       httpsgithubcommarketplaceactionsmetricsembed   
2  httpsgithubcommarketplaceactionsyqportableyaml...   
3        httpsgithubcommarketplaceactionssuperlinter   
4  h

Data Quality


In [ ]:
import pandas as pd
import csv
import re


df = pd.read_csv('products_info.csv')

print("Checking for missing values...")
missing_values = df.isnull().sum()
print(missing_values)

df = df.dropna(subset=['name', 'url', 'description', 'page_number'])

print("Checking for duplicate entries...")
df_duplicates = df[df.duplicated(subset=['url'], keep='first')]  # Keep the first occurrence of each duplicate
if not df_duplicates.empty:
    print(f"Found {len(df_duplicates)} duplicate entries. Removing duplicates.")
    df = df.drop_duplicates(subset=['url'], keep='first')
else:
    print("No duplicates found.")

print("Validating URLs...")
valid_urls = df['url'].apply(lambda x: bool(re.match(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', x)))
invalid_urls = df[~valid_urls]  # Get rows with invalid URLs
if not invalid_urls.empty:
    print(f"Found {len(invalid_urls)} rows with invalid URLs. Removing them.")
    df = df[valid_urls]  # Remove invalid URLs
else:
    print("All URLs are valid.")

print("Validating 'page_number'...")
df['page_number'] = pd.to_numeric(df['page_number'], errors='coerce')  # Convert to numeric, invalid values will become NaN
df = df.dropna(subset=['page_number'])  # Remove rows where 'page_number' is NaN


df.to_csv('products_info_cleaned.csv', index=False)

print("\nCleaned data sample:")
print(df.head())

print("\nData Quality Check Summary:")
print(f"Total rows after cleaning: {len(df)}")
print(f"Missing values after cleaning: {df.isnull().sum()}")


Checking for missing values...
name           0
url            0
description    0
page_number    0
dtype: int64
Checking for duplicate entries...
Found 7 duplicate entries. Removing duplicates.
Validating URLs...
All URLs are valid.
Validating 'page_number'...

Cleaned data sample:
                           name  \
0                TruffleHog OSS   
1                 Metrics embed   
2  yq - portable yaml processor   
3                  Super-Linter   
4        Gosec Security Checker   

                                                 url  \
0  https://github.com/marketplace/actions/truffle...   
1  https://github.com/marketplace/actions/metrics...   
2  https://github.com/marketplace/actions/yq-port...   
3  https://github.com/marketplace/actions/super-l...   
4  https://github.com/marketplace/actions/gosec-s...   

                                         description  page_number  
0              Scan Github Actions with TruffleHog\n            1  
1  An infographics generator with

#Question 5 (20 points)

PART 1:
Web Scrape  tweets from Twitter using the Tweepy API, specifically targeting hashtags related to subtopics (machine learning or artificial intelligence.)
The extracted data includes the tweet ID, username, and text.

Part 2:
Perform data cleaning procedures

A final data quality check ensures the completeness and consistency of the dataset. The cleaned data is then saved into a CSV file for further analysis.


**Note**

1.   Follow tutorials provided in canvas to obtain api keys. Use ChatGPT to get the code. Make sure the file is downloaded and saved.
2.   Make sure you divide GPT code as shown in tutorials, dont make multiple requestes.


In [ ]:
pip install tweepy

In [ ]:
!pip install tweepy
import tweepy
import pandas as pd


bearer_token = 'AAAAAAAAAAAAAAAAAAAAALilzQEAAAAAkcarJ3HmwmE6Nj2bp5ny59hPVik%3DNWgabd8iSYP4PiWmkvlWPyp8l341IsHO5ibljBVR6w1adoJkCw'

In [ ]:

client = tweepy.Client(bearer_token=bearer_token)

query = "Machine Learning"
tweets = client.search_recent_tweets(query=query, tweet_fields=["created_at", "text", "author_id"], max_results=100)

tweet_data = []

if tweets.data:
    for tweet in tweets.data:
        tweet_data.append({
            'tweet_id': tweet.id,
            'author_id': tweet.author_id,
            'text': tweet.text
        })

df = pd.DataFrame(tweet_data)
print(df)


df.to_csv("tweets_llm.csv", index=False)

               tweet_id            author_id  \
0   1892423053352595833   961934520202158080   
1   1892422928517808322  1381957245915762690   
2   1892422681623957983   788898706586275840   
3   1892422602104143897  1855984825514991616   
4   1892422519321174453             53103453   
..                  ...                  ...   
95  1892416092875763890  1867900705157292032   
96  1892416081026855142            318906105   
97  1892416053004632377  1757830802979565568   
98  1892416050270023899  1784204075720212480   
99  1892416047581483046  1858996603673014272   

                                                 text  
0   RT @SakanaAILabs: Introducing The AI CUDA Engi...  
1   Need help with #Assignment Essay #Homework wri...  
2   Become a TDS contributor 📢 Share your insights...  
3   @refaee_abd we could trace how babylonian astr...  
4   RT @SakanaAILabs: Introducing The AI CUDA Engi...  
..                                                ...  
95  RT @DannyOfCrypto: ➥ How Zk

In [ ]:
df = pd.read_csv("/content/tweets_llm.csv")

In [ ]:
missing_values = df.isnull().sum()
print(f"Missing values before cleaning:\n{missing_values}\n")


duplicates = df[df.duplicated()]
print(f"Duplicates before cleaning:\n{duplicates}\n")
df = df.drop_duplicates()

df['text'] = df['text'].str.strip()
df['text'] = df['text'].str.replace('\n', ' ', regex=False)


if not df['tweet_id'].apply(lambda x: isinstance(x, int)).all():
    print("Warning: Some tweet_id values are not integers.")
if not df['author_id'].apply(lambda x: isinstance(x, int)).all():
    print("Warning: Some author_id values are not integers.")


missing_values_after = df.isnull().sum()
print(f"Missing values after cleaning:\n{missing_values_after}\n")
print("Cleaned DataFrame:")
print(df)


output_file = 'twitter_cleaned_data.csv'
df.to_csv(output_file, index=False)
print(f"Cleaned data saved to {output_file}")


Missing values before cleaning:
tweet_id     0
author_id    0
text         0
dtype: int64

Duplicates before cleaning:
Empty DataFrame
Columns: [tweet_id, author_id, text]
Index: []

Missing values after cleaning:
tweet_id     0
author_id    0
text         0
dtype: int64

Cleaned DataFrame:
               tweet_id            author_id  \
0   1892423053352595833   961934520202158080   
1   1892422928517808322  1381957245915762690   
2   1892422681623957983   788898706586275840   
3   1892422602104143897  1855984825514991616   
4   1892422519321174453             53103453   
..                  ...                  ...   
95  1892416092875763890  1867900705157292032   
96  1892416081026855142            318906105   
97  1892416053004632377  1757830802979565568   
98  1892416050270023899  1784204075720212480   
99  1892416047581483046  1858996603673014272   

                                                 text  
0   RT @SakanaAILabs: Introducing The AI CUDA Engi...  
1   Need help with 

In [18]:
import pandas as pd
import re

df = pd.read_csv("/content/twitter_cleaned_data.csv")

def clean(text):
    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    # Remove mentions
    text = re.sub(r"@\w+", "", text)
    # Remove non-alphanumeric characters
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    # Convert to lowercase
    text = text.lower()
    # Remove extra spaces
    text = " ".join(text.split())
    return text


df["cleaned_tweets"] = df["text"].apply(clean)


print(df[["text", "cleaned_tweets"]].head())

                                                text  \
0  RT @SakanaAILabs: Introducing The AI CUDA Engi...   
1  Need help with #Assignment Essay #Homework wri...   
2  Become a TDS contributor 📢 Share your insights...   
3  @refaee_abd we could trace how babylonian astr...   
4  RT @SakanaAILabs: Introducing The AI CUDA Engi...   

                                      cleaned_tweets  
0  rt introducing the ai cuda engineer an agentic...  
1  need help with assignment essay homework write...  
2  become a tds contributor share your insights o...  
3  we could trace how babylonian astronomers basi...  
4  rt introducing the ai cuda engineer an agentic...  


# Mandatory Question

Provide your thoughts on the assignment. What did you find challenging, and what aspects did you enjoy? Your opinion on the provided time to complete the assignment.

For this assignment need to know how to use models of web scrapping for the differnt web urls, and APIS. I worked on  nearly 4 hours to complete the assignment. Extracting the  data from the web urls by using Beautifulsoup library for all models make me diffcult to downlaod the parser.html file.


# Write your response below
Fill out survey and provide your valuable feedback.

https://docs.google.com/forms/d/e/1FAIpQLSd_ObuA3iNoL7Az_C-2NOfHodfKCfDzHZtGRfIker6WyZqTtA/viewform?usp=dialog